# Solar Dynamics Observatory

Single timeseries of EUV data

In [ ]:
import intake
import hvplot.xarray

In [ ]:
cat = intake.open_catalog("https://github.com/fsspec/kerchunk/raw/main/"
                          "examples/intake_catalog.yml")

In [ ]:
ds = cat.SDO.to_dask()

In [ ]:
import numpy as np
ds = ds.assign_coords(x=np.arange(4096), y=np.arange(4096))

In [ ]:
ds

In [ ]:
ds.nbytes / 2**30. # GB

### Viz

In [ ]:
import numpy as np
ds["094"].hvplot.image('x', 'y', colormap="fire", xaxis=False, yaxis=False, 
                height=500, width=500, colorbar=False, clim=(0, 50))